For this workshop, you need:

* An Azure Machine Learning workspace. 
* The Azure Machine Learning Python SDK v2 installed. 

To install the SDK you can either,

Create a compute instance, which already has installed the latest AzureML Python SDK and is pre-configured for ML workflows.

Use the followings commands to install Azure ML Python SDK v2:

```bash
conda activate <virtual_env_name>
pip install azure-ai-ml==1.0.0
```

If you're using a virtual env, make sure to install the sdk inside the virtual env.

The virtual environment for sdkv2 on Azure Notebooks is called `azureml_py310_sdkv2`.


## Connect to ML Client

To connect to a workspace, you need to provide a subscription, resource group and workspace name. These details are used in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace.

In the following example, the default Azure authentication is used along with the default workspace configuration or from any `config.json` file you might have copied into the folders structure. If no `config.json` is found, then you need to manually introduce the subscription_id, resource_group and workspace when creating `MLClient`.

```python
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AzureML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AZUREML_WORKSPACE_NAME>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
```


In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Add config.json file to the workspace
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential, path="config.json")

Found the config file in: /config.json


# Model Training

## 1. Create Managed Compute

A compute is a designated compute resource where you run your job or host your endpoint. Azure Machine learning supports the following types of compute:

- **Compute instance** - a fully configured and managed development environment in the cloud. You can use the instance as a training or inference compute for development and testing. It's similar to a virtual machine on the cloud.

- **Compute cluster** - a managed-compute infrastructure that allows you to easily create a cluster of CPU or GPU compute nodes in the cloud.

- **Inference cluster** - used to deploy trained machine learning models to Azure Kubernetes Service. You can create an Azure Kubernetes Service (AKS) cluster from your Azure ML workspace, or attach an existing AKS cluster.

- **Attached compute** - You can attach your own compute resources to your workspace and use them for training and inference.

You can create a compute using the Studio, the cli and the sdk.

<hr>

We can create a **compute instance** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../../imgs/create_compute_instance.png" width = "700px" alt="Create Compute Instance cli vs sdk">
</center>


<hr>

We can create a **compute cluster** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../../imgs/create_compute_cluster.png" width = "700px" alt="Create Compute Instance cli vs sdk">
</center>


Let's create a managed compute cluster for the training workload.

In [2]:
from azure.ai.ml.entities import AmlCompute

my_cluster = AmlCompute(
    name="cpu-cluster",
    type="amlcompute", 
    size="STANDARD_DS3_V2", 
    min_instances=0, 
    max_instances=4,
    location="westeurope", 	
)

ml_client.compute.begin_create_or_update(my_cluster)


## 2. Register Data Asset

**Datastore** - Azure Machine Learning Datastores securely keep the connection information to your data storage on Azure, so you don't have to code it in your scripts.

An Azure Machine Learning datastore is a **reference** to an **existing** storage account on Azure. The benefits of creating and using a datastore are:
* A common and easy-to-use API to interact with different storage type. 
* Easier to discover useful datastores when working as a team.
* When using credential-based access (service principal/SAS/key), the connection information is secured so you don't have to code it in your scripts.

Supported Data Resources: 

* Azure Storage blob container
* Azure Storage file share
* Azure Data Lake Gen 1
* Azure Data Lake Gen 2
* Azure SQL Database 
* Azure PostgreSQL Database
* Azure MySQL Database

It is not a requirement to use Azure Machine Learning datastores - you can use storage URIs directly assuming you have access to the underlying data.

You can create a datastore using the Studio, the cli and the sdk.

<hr>

We can create a **datastore** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../../imgs/create_datastore.png" width = "700px" alt="Create Datastore cli vs sdk">
</center>



**Data asset** - Create data assets in your workspace to share with team members, version, and track data lineage.

By creating a data asset, you create a reference to the data source location, along with a copy of its metadata. 

The benefits of creating data assets are:

* You can **share and reuse data** with other members of the team such that they do not need to remember file locations.
* You can **seamlessly access data** during model training (on any supported compute type) without worrying about connection strings or data paths.
* You can **version** the data.

<hr>

We can create a **data asset** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../../imgs/create_data_asset.png" width = "700px" alt="Create Data Asset cli vs sdk">
</center>

In [3]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_data = Data(
    path="../../../data/taxi-data.csv",
    type="uri_file",
    description="Taxi dataset",
    name="taxi-data"
)
ml_client.data.create_or_update(my_data)

Uploading taxi-data.csv (< 1 MB): 100%|██████████| 1.21M/1.21M [00:00<00:00, 22.0MB/s]




Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'taxi-data', 'description': 'Taxi dataset', 'tags': {}, 'properties': {}, 'id': '/subscriptions/d5eebdae-19f7-4198-a12e-04fdec08b615/resourceGroups/seenergyforecast/providers/Microsoft.MachineLearningServices/workspaces/seenergyforecast/data/taxi-data/versions/1', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/maggiemhanna-ws/code/Users/maggiemhanna/mlops-v2-training/mlops/azureml/sdk', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f81486e25f0>, 'serialize': <msrest.serialization.Serializer object at 0x7f81486e2770>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/d5eebdae-19f7-4198-a12e-04fdec08b615/resourcegroups/seenergyforecast/workspaces/seenergyforecast/datastores/workspaceblobstore/paths/LocalUpload/9292ec840b5

## 3. Register Train Environment

Azure Machine Learning environments define the execution environments for your **jobs** or **deployments** and encapsulate the dependencies for your code. 

Azure ML uses the environment specification to create the Docker container that your **training** or **scoring code** runs in on the specified compute target.

Create an environment from a
* conda specification
* Docker image
* Docker build context

There are two types of environments in Azure ML: **curated** and **custom environments**. Curated environments are predefined environments containing popular ML frameworks and tooling. Custom environments are user-defined.

<hr>

We can register an **environment** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../../imgs/create_environment.png" width = "700px" alt="Create Environment cli vs sdk">
</center>

In [4]:
from azure.ai.ml.entities import Environment

my_environment = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="../../../data-science/environment/train-conda.yml",
    name="taxi-train-env",
    description="Environment created from a Docker image plus Conda environment to train taxi model.",
)

ml_client.environments.create_or_update(my_environment)

Environment({'is_anonymous': False, 'auto_increment_version': False, 'name': 'taxi-train-env', 'description': 'Environment created from a Docker image plus Conda environment to train taxi model.', 'tags': {}, 'properties': {}, 'id': '/subscriptions/d5eebdae-19f7-4198-a12e-04fdec08b615/resourceGroups/seenergyforecast/providers/Microsoft.MachineLearningServices/workspaces/seenergyforecast/environments/taxi-train-env/versions/1', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/maggiemhanna-ws/code/Users/maggiemhanna/mlops-v2-training/mlops/azureml/sdk', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f81486e2c80>, 'serialize': <msrest.serialization.Serializer object at 0x7f81486e1f00>, 'version': '1', 'latest_version': None, 'conda_file': {'channels': ['defaults', 'anaconda', 'conda-forge'], 'dependencies': ['python=3.7.5', 'pip', {'pip': ['azureml-mlflow==1.38.0', 'azure-ai-ml==1.0.0', 'pyarrow==10.0.0', 'ruamel.

## 4. Create Pipeline Job

**AML Job**:

Azure ML provides several ways to train your models, from code-first solutions to low-code solutions:

* Azure ML supports script files in python, R, Java, Julia or C#. All you need to learn is YAML format and command lines to use Azure ML.

* Distributed Training: AML supports integrations with popular frameworks, PyTorch and TensorFlow. Both frameworks employ data parallelism & model parallelism for distributed training.

* Automated ML - Train models without extensive data science or programming knowledge.

* Designer - drag and drop web-based UI.

<hr>

We can submit a **job** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../../imgs/create_job.png" width = "700px" alt="Create Job cli vs sdk">
</center>

<br>
    
**AML Pipelines**:

An AML pipeline is an independently executable workflow of a complete machine learning task. It helps standardizing the best practices of producing a machine learning model: The core of a machine learning pipeline is to split a complete machine learning task into a multistep workflow. Each step is a manageable component that can be developed, optimized, configured, and automated individually. 

<hr>

We can submit a **pipeline job** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../../imgs/create_pipeline.png" width = "700px" alt="Create Pipeline cli vs sdk">
</center>

In [5]:
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import Input, Output, command
from azure.ai.ml.constants import AssetTypes, InputOutputModes


# 1. Define components

prep_data = command( 
    name="prep_data",
    display_name="prep-data",
    code="../../../data-science/src/prep",
    command="python prep.py \
            --raw_data ${{inputs.raw_data}} \
            --train_data ${{outputs.train_data}}  \
            --val_data ${{outputs.val_data}} \
            --test_data ${{outputs.test_data}} \
            --enable_monitoring ${{inputs.enable_monitoring}} \
            --table_name ${{inputs.table_name}}",
    environment="taxi-train-env@latest",
    inputs={
        "raw_data": Input(type="uri_file"),
        "enable_monitoring": Input(type="string"),
        "table_name": Input(type="string")
        },
    outputs={
        "train_data": Output(type="uri_folder"),
        "val_data": Output(type="uri_folder"),
        "test_data": Output(type="uri_folder"),
        }
)

train_model = command( 
    name="train_model",
    display_name="train-model",
    code="../../../data-science/src/train",
    command="python train.py \
            --train_data ${{inputs.train_data}} \
            --model_output ${{outputs.model_output}}",
    environment="taxi-train-env@latest",
    inputs={"train_data": Input(type="uri_folder")},
    outputs={"model_output": Output(type="uri_folder")}
)

evaluate_model = command(
    name="evaluate_model",
    display_name="evaluate-model",
    code="../../../data-science/src/evaluate",
    command="python evaluate.py \
            --model_name ${{inputs.model_name}} \
            --model_input ${{inputs.model_input}} \
            --test_data ${{inputs.test_data}} \
            --evaluation_output ${{outputs.evaluation_output}}",
    environment="taxi-train-env@latest",
    inputs={
        "model_name": Input(type="string"),
        "model_input": Input(type="uri_folder"),
        "test_data": Input(type="uri_folder")
        },
    outputs={
        "evaluation_output": Output(type="uri_folder")
        }
)

register_model = command(
    name="register_model",
    display_name="register-model",
    code="../../../data-science/src/register",
    command="python register.py \
            --model_name ${{inputs.model_name}} \
            --model_path ${{inputs.model_path}} \
            --evaluation_output ${{inputs.evaluation_output}} \
            --model_info_output_path ${{outputs.model_info_output_path}}",
    environment="taxi-train-env@latest",
    inputs={
        "model_name": Input(type="string"),
        "model_path": Input(type="uri_folder"),
        "evaluation_output": Input(type="uri_folder")
        },
    outputs={
        "model_info_output_path": Output(type="uri_folder")
        }
)

# 2. Construct pipeline
@pipeline()
def taxi_training_pipeline(raw_data, enable_monitoring, table_name):
    
    prep = prep_data(
        raw_data=raw_data,
        enable_monitoring=enable_monitoring, 
        table_name=table_name
    )

    train = train_model(
        train_data=prep.outputs.train_data
    )

    evaluate = evaluate_model(
        model_name="taxi-model",
        model_input=train.outputs.model_output,
        test_data=prep.outputs.test_data
    )


    register = register_model(
        model_name="taxi-model",
        model_path=train.outputs.model_output,
        evaluation_output=evaluate.outputs.evaluation_output
    )

    return {
        "pipeline_job_train_data": prep.outputs.train_data,
        "pipeline_job_test_data": prep.outputs.test_data,
        "pipeline_job_trained_model": train.outputs.model_output,
        "pipeline_job_score_report": evaluate.outputs.evaluation_output,
    }


pipeline_job = taxi_training_pipeline(
    Input(type="uri_file", path="taxi-data@latest"), "false", "taximonitoring"
)

# set pipeline level compute
pipeline_job.settings.default_compute = "cpu-cluster"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

In [6]:
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="taxi-training"
)
pipeline_job

Uploading prep (0.01 MBs): 100%|██████████| 9644/9644 [00:00<00:00, 173249.16it/s]


Uploading train (0.01 MBs): 100%|██████████| 10615/10615 [00:00<00:00, 205227.83it/s]


Uploading evaluate (0.01 MBs): 100%|██████████| 12215/12215 [00:00<00:00, 399356.33it/s]


Uploading register (0.0 MBs): 100%|██████████| 4568/4568 [00:00<00:00, 108108.18it/s]




Experiment,Name,Type,Status,Details Page
taxi-training,sincere_peach_0n96n0j3zq,pipeline,Preparing,Link to Azure Machine Learning studio
